In [2]:
import sys
!{sys.executable} -m pip install -r /home/mle-user/mle_projects/mle-project-sprint-1-v001/requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached Babel-2.13.1-py3-none-any.whl (10.1 MB)
  Using cached cachecontrol-0.13.1-py3-none-any.whl (22 kB)
  Using cached ConfigUpdater-3.2-py2.py3-none-any.whl (34 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl (9.6 kB)
  Using cached Flask_AppBuilder-4.3.6-py3-none-any.whl (1.7 MB)
  Using cached Flask_Babel-2.0.0-py3-none-any.whl (9.3 kB)
  Using cached Flask_Caching-2.1.0-py3-none-any.whl (28 kB)
  Using cached Flask_JWT_Extended-4.5.3-py2.py3-none-any.whl (22 kB)
  Using cached Flask_Limiter-3.5.0-py3-none-any.whl (28 kB)
  Using cached Flask_Login-0.6.3-py3-none-any.whl (17 kB)
  Using cached Flask_SQLAlchemy-2.5.1-py2.py3-none-any.whl (17 kB)
  Using cached flask_session-0.5.0-py3-none-any.whl (7.2 kB)
  Using cached flask_wtf-1.2.1-py3-none-any.whl (12 kB)
  Using cached Flask-2.2.5-py3-none-any.whl (101 kB)
  Using cached GitPython-3.1.42-py3-none-any.whl (195 kB)
  Using cached Ji

In [3]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine

In [4]:
# подгружаем .env
load_dotenv()

True

In [5]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [6]:
# Создадим соединения
src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

### Просмотр и объединение таблиц

In [42]:
TABLE = 'buildings'
SQL = f'select * from {TABLE}'
buildings = pd.read_sql(SQL, dst_conn)

TABLE = 'flats'
SQL = f'select * from {TABLE}'
flats = pd.read_sql(SQL, dst_conn)



In [43]:
flats.head()

,id,floor,is_apartment,kitchen_area,living_area,rooms,studio,total_area,price,building_id
0,0,9,False,9.9,19.900000,1,False,35.099998,9500000,6220
1,1,7,False,0.0,16.600000,1,False,43.000000,13500000,18012
2,2,9,False,9.0,32.000000,2,False,56.000000,13500000,17821
3,3,1,False,10.1,43.099998,3,False,76.000000,20000000,18579
4,4,3,False,3.0,14.000000,1,False,24.000000,5200000,9293


In [44]:
buildings.head()

,id,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,6220,1965,6,55.717113,37.781120,2.64,84,12,True
1,18012,2001,2,55.794849,37.608013,3.00,97,10,True
2,17821,2000,4,55.740040,37.761742,2.70,80,10,True
3,18579,2002,4,55.672016,37.570877,2.64,771,17,True
4,9293,1971,1,55.808807,37.707306,2.60,208,9,True


In [45]:
buildings_full = buildings.merge(flats, on='id', how='left')
print(buildings_full.head())
print(buildings_full.isna().sum())

      id  build_year  building_type_int   latitude  longitude  ceiling_height  \
0   6220        1965                  6  55.717113  37.781120            2.64   
1  18012        2001                  2  55.794849  37.608013            3.00   
2  17821        2000                  4  55.740040  37.761742            2.70   
3  18579        2002                  4  55.672016  37.570877            2.64   
4   9293        1971                  1  55.808807  37.707306            2.60   

   flats_count  floors_total  has_elevator  floor  is_apartment  kitchen_area  \
0           84            12          True      2         False          10.2   
1           97            10          True     16         False          12.8   
2           80            10          True      8         False           8.5   
3          771            17          True     10         False           0.0   
4          208             9          True      5         False           8.2   

   living_area  rooms  stu

In [46]:
print(buildings_full.dtypes)

id                     int64
build_year             int64
building_type_int      int64
latitude             float64
longitude            float64
ceiling_height       float64
flats_count            int64
floors_total           int64
has_elevator            bool
floor                  int64
is_apartment            bool
kitchen_area         float64
living_area          float64
rooms                  int64
studio                  bool
total_area           float64
price                  int64
building_id            int64
dtype: object


In [47]:
buildings_full.describe()

,id,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,floor,kitchen_area,living_area,rooms,total_area,price,building_id
count,24620.00000,24620.000000,24620.000000,24620.000000,24620.000000,24620.000000,24620.000000,24620.000000,24620.000000,24620.000000,24620.000000,24620.000000,24620.000000,2.462000e+04,24620.000000
mean,12310.50000,1981.377295,3.153656,55.738892,37.593816,2.753557,184.882575,12.441024,7.280788,8.761977,31.278825,2.097238,59.859355,1.759778e+07,13582.866328
std,7107.32615,22.657856,1.636435,0.101094,0.145408,0.277912,158.427509,6.271325,5.501710,4.761168,22.070693,0.970318,37.104542,3.788168e+07,6826.143476
min,1.00000,1901.000000,0.000000,55.211460,36.864372,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,11.500000,3.000000e+03,3.000000
25%,6155.75000,1965.000000,1.000000,55.668725,37.499031,2.640000,80.000000,8.000000,3.000000,6.100000,19.000000,1.000000,39.000000,8.977500e+06,8280.000000
50%,12310.50000,1978.000000,4.000000,55.742020,37.588400,2.640000,139.000000,12.000000,6.000000,8.600000,30.000000,2.000000,52.000000,1.170000e+07,13653.000000
75%,18465.25000,2002.000000,4.000000,55.811775,37.698833,2.900000,242.000000,16.000000,10.000000,10.000000,41.000000,3.000000,67.000000,1.600000e+07,19619.250000
max,24620.00000,2023.000000,6.000000,56.011032,37.946411,27.000000,4455.000000,99.000000,55.000000,100.000000,700.000000,20.000000,960.299988,2.551801e+09,24619.000000


In [48]:
buildings_full[buildings_full['living_area'] > buildings_full['total_area']]

,id,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,floor,is_apartment,kitchen_area,living_area,rooms,studio,total_area,price,building_id


In [49]:
buildings_full[buildings_full.duplicated(subset=list(buildings_full.drop(columns=['id']).columns))]

,id,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,floor,is_apartment,kitchen_area,living_area,rooms,studio,total_area,price,building_id


In [52]:
buildings_full.shape

(24620, 18)

In [53]:
buildings_full[(buildings_full['living_area'] + buildings_full['kitchen_area']) > buildings_full['total_area']]

,id,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,floor,is_apartment,kitchen_area,living_area,rooms,studio,total_area,price,building_id
3020,2104,1958,1,55.672676,37.594540,3.20,58,5,False,3,False,7.300000,37.099998,1,False,39.500000,12200000,22205
4414,12976,1980,4,55.646015,37.723007,2.70,355,9,True,2,False,6.000000,28.000000,1,False,33.000000,7180000,3210
4711,4360,1962,4,55.666042,37.551556,2.64,80,5,False,8,False,10.200000,38.900002,1,False,45.000000,8100000,14704
5281,1152,1951,1,55.773575,37.629520,3.20,137,9,True,4,False,6.000000,66.000000,3,False,67.000000,35000000,687
9305,2207,1958,1,55.726944,37.638359,2.70,45,5,False,5,False,16.900000,29.299999,2,False,36.799999,13900000,11701
9426,14055,1984,4,55.603359,37.708069,2.64,128,16,True,15,False,18.800000,152.300000,6,False,154.600000,17000000,21795
9587,3935,1961,4,55.720879,37.609344,2.64,48,5,False,2,False,9.000000,55.000000,2,False,59.299999,9500000,22141
9616,1631,1956,1,55.724945,37.771057,3.00,48,5,False,7,False,12.000000,53.200001,3,False,65.199997,16990000,22656
10545,22022,2012,2,55.594223,37.341045,2.70,471,17,True,3,False,33.200001,45.000000,2,False,78.199997,43472036,23738
11380,498,1928,1,55.764492,37.608040,3.20,69,7,True,5,False,19.500000,19.500000,1,False,32.000000,8050000,4645
